In [2]:
import sys
import os

# Determine the current directory and add the parent directory to the system path
current_dir = os.path.abspath('.')
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

Parent directory added to sys.path: /home/zhang0311/FormulaLLMPY


In [4]:
from formulallm.agents import executor_agent, fixer_agent, interpreter_agent, userproxy_agent
from autogen import GroupChat, GroupChatManager
from formula.formula_program import *

In [5]:
local_llm_config={
    "config_list": [
        {
            "model": "NotRequired", # Loaded with LiteLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:4000"  # Your LiteLLM URL
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

In [6]:
groupchat = GroupChat(
    agents=[userproxy_agent, fixer_agent, executor_agent, interpreter_agent],
    messages=[],
    max_round=10,
    send_introductions=True,
    allowed_or_disallowed_speaker_transitions = {
        fixer_agent: [userproxy_agent, executor_agent],
        executor_agent: [userproxy_agent]
    },
    speaker_transitions_type="allowed"
)


In [ ]:
manager = GroupChatManager(
    groupchat=groupchat, llm_config=local_llm_config
)

In [ ]:
task = ''

In [ ]:

groupchat_result = userproxy_agent.initiate_chat(
    manager,
    message=task,
)